In [4]:
import pandas as pd
from PIL import Image, ImageEnhance, ImageFont, ImageDraw
import os
import math

DATASTORE_PATH = "/datastore/AGILE/BELToutput/LDP_Sebastian"

# This script expects the following headers in the below csv: img,k,group_pred
INPUT_CSV_PATH = "/birl2/users/sch923/Thesis/Lentil/pattern_5000_new.csv"

SUB_IMG_X_Y = 200
IMGS_PER_ROW = 10
COLLAGE_X_Y = SUB_IMG_X_Y * IMGS_PER_ROW

OUTDIR = "/birl2/users/sch923/Thesis/Lentil/Collages_new_5000"

FONT_PATH = "/birl2/users/sch923/times-new-roman.ttf"

In [2]:
# Cell used to grab all image locals, kgroups, and prediction groups from csv

predictions_df = pd.read_csv(INPUT_CSV_PATH)            

In [5]:
# Creates collage

max_k_list = predictions_df['k'].unique().tolist()
font = ImageFont.truetype(FONT_PATH, 64)
for max_k in max_k_list:
    # Filters predictions to only include the current max_k
    max_k_filtered_df = predictions_df[predictions_df['k'] == max_k]
    for pred in range(0, max_k):
        filtered_df = max_k_filtered_df[max_k_filtered_df['group_pred'] == pred]
        per_row = min(IMGS_PER_ROW, int(math.sqrt(len(filtered_df))))
        print(f"IMGS_PER_ROW: {IMGS_PER_ROW}, int(math.sqrt(len(filtered_df))): {int(math.sqrt(len(filtered_df)))}")
        collage_x_y = SUB_IMG_X_Y * per_row
        pred_sample_df = filtered_df.sample(n=per_row**2, random_state=1, replace=False) # Random state was selected at random (manually)
        collage = Image.new("RGBA", (collage_x_y,collage_x_y))
        img_path_list = pred_sample_df['img'].tolist()
        for x in range(0, per_row):
            for y in range(0, per_row):
                img = Image.open(img_path_list.pop())
                img = img.resize((SUB_IMG_X_Y,SUB_IMG_X_Y))
                collage.paste(img, (x*SUB_IMG_X_Y,y*SUB_IMG_X_Y))
        
        
        # Label the image with interesting metadata and save
        draw = ImageDraw.Draw(collage)
        draw.text((0, 0),f" Total Groups = {max_k}, Group Prediction = {pred+1}",(255,255,255), font=font)
        # collage.show()
        collage_name = f"totalk{max_k}_predk{pred+1}_lentil_collage_5000.png"
        collage.save(os.path.join(OUTDIR,collage_name))

IMGS_PER_ROW: 10, len(filtered_df): 449
IMGS_PER_ROW: 10, len(filtered_df): 3586
IMGS_PER_ROW: 10, len(filtered_df): 3268
IMGS_PER_ROW: 10, len(filtered_df): 597
IMGS_PER_ROW: 10, len(filtered_df): 170
IMGS_PER_ROW: 10, len(filtered_df): 536
